In [ ]:
%pip install rdflib

In [ ]:
import google
import os

# prompt: mount drive here to read files from the folder "My Drive > Colab_Notebooks > Formal_Ontology_of_Mathematics"
google.colab.drive.mount('/content/drive')

os.chdir("/content/drive/My Drive/Colab_Notebooks/Formal_Ontology_of_Mathematics")
!ls
#Now you can access files in this directory.  For example:
#with open("my_file.txt", "r") as f:
#    file_contents = f.read()
#    print(file_contents)


In [ ]:
import pandas as pd
import os
import re
import rdflib

import modules.utils as utils
import modules.tbox as tbox
import modules.concepts as concepts

# common IRIs
rdf_type = rdflib.RDF.type
rdfs_label = rdflib.RDFS.label
rdfs_subclassof = rdflib.RDFS.subClassOf
skos_prefLabel = rdflib.SKOS.prefLabel
skos_altLabel = rdflib.SKOS.altLabel
owl_class = rdflib.OWL.Class
owl_individual = rdflib.OWL.NamedIndividual
owl_object_property = rdflib.OWL.ObjectProperty
owl_data_property = rdflib.OWL.DatatypeProperty
owl_annotation_property = rdflib.OWL.AnnotationProperty

# classes IRIs
concept_class = utils.create_iri("Concept", namespace="https://www.foom.com/core")
domain_class = utils.create_iri("Domain", namespace="https://www.foom.com/core")
enumeration_class = utils.create_iri("Enumeration", namespace="https://www.foom.com/core")
magnitude_class = utils.create_iri("Magnitude", namespace="https://www.foom.com/core")
operation_class = utils.create_iri("Operation", namespace="https://www.foom.com/core")
operation_instance_class = utils.create_iri("Operation instance", namespace="https://www.foom.com/core")
proposition_type_class = utils.create_iri("Proposition type", namespace="https://www.foom.com/core")
relation_class = utils.create_iri("Relation", namespace="https://www.foom.com/core")
relation_instance_class = utils.create_iri("Relation instance", namespace="https://www.foom.com/core")

# object properties IRIs
elements_book_1 = utils.create_iri("Document: Elements Book 1", namespace="https://www.foom.com/core")
proposition_type_construction = utils.create_iri("Proposition type: Construction", namespace="https://www.foom.com/core")
proposition_type_theorem = utils.create_iri("Proposition type: Theorem", namespace="https://www.foom.com/core")

refers_to = utils.create_iri("refers to", namespace="https://www.foom.com/core")
definition_refers_to = utils.create_iri("definition refers to", namespace="https://www.foom.com/core")

has_conceptual_component = utils.create_iri("has conceptual component", namespace="https://www.foom.com/core")
is_conceptual_component_of = utils.create_iri("is conceptual component of", namespace="https://www.foom.com/core")

is_sub-concept_of = utils.create_iri("is sub-concept of", namespace="https://www.foom.com/core")
is_super-concept_of = utils.create_iri("is super-concept of", namespace="https://www.foom.com/core")

is_used_in_definition_of = utils.create_iri("is used in definition of", namespace="https://www.foom.com/core")
is_defined_in = utils.create_iri("is defined in", namespace="https://www.foom.com/core")

contains_definition_of = utils.create_iri("contains_definition_of", namespace="https://www.foom.com/core")
is_used_in = utils.create_iri("is used in", namespace="https://www.foom.com/core")

has_definition = utils.create_iri("has definition", namespace="https://www.foom.com/core")
defines = utils.create_iri("defines", namespace="https://www.foom.com/core")

has_subject = utils.create_iri("has subject", namespace="https://www.foom.com/core")
has_predicate = utils.create_iri("has predicate", namespace="https://www.foom.com/core")
has_object = utils.create_iri("has object", namespace="https://www.foom.com/core")

has_domain = utils.create_iri("has domain", namespace="https://www.foom.com/core")
has_range = utils.create_iri("has_range", namespace="https://www.foom.com/core")

In [ ]:

# initialize graph
kg = rdflib.Graph()

# tbox and foundational triples
ontology_items = {
    # classes
    ("Concept", owl_class),
    ("Common notion", owl_class),
    ("Definition", owl_class),
    ("Document", owl_class),
    ("Domain", owl_class),
    ("Enumeration", owl_class),
    ("Magnitude", owl_class),
    ("Operation", owl_class),
    ("Operation instance", owl_class),
    ("Postulate", owl_class),
    ("Proposition", owl_class),
    ("Proposition type", owl_class),
    ("Relation", owl_class),
    ("Relation instance", owl_class),
    ("Statement", owl_class),

    # object properties
    ("refers to", owl_object_property),
    ("definition refers to", owl_object_property),

    ("has conceptual component", owl_object_property),
    ("is conceptual component of", owl_object_property),

    ("is sub-concept of", owl_object_property),
    ("is super-concept of", owl_object_property),

    ("is used in definition of", owl_object_property),
    ("is defined in", owl_object_property),

    ("contains_definition_of", owl_object_property),
    ("is used in", owl_object_property),

    ("has definition", owl_object_property),
    ("defines", owl_object_property),

    ("has subject", owl_object_property),
    ("has predicate", owl_object_property),
    ("has object", owl_object_property),

    ("has domain", owl_object_property),
    ("has_range", owl_object_property)
}
triples = {
    (elements_book_1, rdf_type, owl_individual),
    (elements_book_1, rdfs_label, rdflib.Literal("Document: Elements Book 1")),
    (elements_book_1, skos_prefLabel, rdflib.Literal("Elements Book 1")),
    (elements_book_1, rdf_type, utils.create_iri("Document", namespace="https://www.foom.com/core")),
    (utils.create_iri("Proposition type", namespace="https://www.foom.com/core"), rdfs_subclassof, enumeration_class),
    (proposition_type_construction, rdf_type, owl_individual),
    (proposition_type_construction, rdf_type, proposition_type_class),
    (proposition_type_construction, rdfs_label, rdflib.Literal("Proposition type: Construction")),
    (proposition_type_construction, skos_prefLabel, rdflib.Literal("Construction")),
    (proposition_type_theorem, rdf_type, owl_individual),
    (proposition_type_theorem, rdf_type, proposition_type_class),
    (proposition_type_theorem, rdfs_label, rdflib.Literal("Proposition type: Theorem")),
    (proposition_type_theorem, skos_prefLabel, rdflib.Literal("Theorem"))
}

kg = tbox.add_tbox(kg, ontology_items, triples)

# abox: concepts
input_file_path = "input/ontology_definitions_concepts.txt"
kg = concepts.main_add_definition_concepts(input_file_path, kg)

# output ontology
utils.output_ontology(kg, "output", "euclid_book1.ttl", "turtle")

In [ ]:
def add_postulates(kg: rdflib.Graph,
                   input_file_path: str) -> rdflib.Graph:
    # lines = utils.read_text_file(input_file_path)
    postulates = pd.read_csv(input_file_path)

    # add postulates
    kg = add_postulate_triples(kg, postulates)

    # add links from postulates to concepts and vice versa
    kg = postulate_links_to_concepts(kg, postulates)

    # add relations
    # kg = add_relations(kg, postulates)

    # add operations

    # add concept hierarchy
    kg = add_concept_hierarchy(kg, postulates)

    return kg

def add_concept_hierarchy(kg: rdflib.Graph,
                          postualtes: pd.DataFrame) -> rdflib.Graph:
    for i in postulates.index:
        hierarchy_items = [hierarchy_item.strip() for hierarchy_item in postulates.at[i, "hierarchy_of_concepts"].split(",")]
        for hierarchy_item in hierarchy_items:
            concepts_pair = hierarchy_item.split("->")
            subconcept = concepts_pair[0].strip()
            superconcept = concepts_pair[1].strip()
            subconcept_iri = utils.create_iri(f"Concept: {subconcept}", namespace="https://www.foom.com/core")
            superconcept_iri = utils.create_iri(f"Concept: {superconcept}", namespace="https://www.foom.com/core")
            kg.add((subconcept_iri, is_sub_concept_of, superconcept_iri))
            kg.add((superconcept_iri, is_super_concept_of, subconcept_iri))
    return kg

def add_relation_triples(kg: rdflib.Graph,
                         relation_iri: rdflib.URIRef,
                         relation_preflabel: str,
                         class_iri: rdflib.Graph,
                         thing_that_refers: rdflib.Graph) -> rdflib.Graph:
    kg.add((relation_iri, rdf_type, owl_individual))
    kg.add((relation_iri, rdf_type, class_iri))
    kg.add((relation_iri, rdfs_label, rdflib.Literal(f"Relation: {relation_preflabel}")))
    kg.add((relation_iri, skos_prefLabel, rdflib.Literal(relation_preflabel)))
    kg.add((thing_that_refers, refers_to, relation_iri))
    kg.add((relation_iri, is_used_in, postulate_iri))
    return kg

def add_relation_concept_triples(kg: rdflib.Graph,
                                 postulates: pd.DataFrame,
                                 column_name: str,
                                 thing_that_refers: rdflib.Graph) -> rdflib.Graph:
    for concept in postulates.at[i, column_name].split(","):
            concept_preflabel = concept.strip()
            concept_iri = utils.create_iri(f"Concept: {concept_preflabel}", namespace="https://www.foom.com/core")
            kg.add((thing_that_refers, refers_to, concept_iri))
            kg.add((concept_iri, is_used_in, thing_that_refers))
    return kg

def add_relations(kg: rdflib.Graph,
                  postulates: pd.DataFrame) -> rdflib.Graph:
    for i in postulates.index:
        postulate = postulates.at[i, "postulate_name"].strip()
        postulate_iri = utils.create_iri(postulate, namespace="https://www.foom.com/core")
        # add relation instances
        relation_preflabel = postulates.at[i, "relations"].strip()
        relation_iri = utils.create_iri(f"Relation: {relation_preflabel}", namespace="https://www.foom.com/core")
        kg = add_relation_triples(kg, postulate_iri, relation_iri, relation_preflabel, relation_instance_class, postulate_iri)
        # add links between relations and concepts
        kg = add_relation_concept_triples(kg, postulates, "relations_concepts", relation_iri)
        # add relation types
        for concept in postulates.at[i, "relations_concepts"].split(","):
            concept_preflabel = concept.strip()
            concept_iri = utils.create_iri(f"Concept: {concept_preflabel}", namespace="https://www.foom.com/core")
            kg.add((relation_iri, refers_to, concept_iri))
            kg.add((concept_iri, is_used_in, relation_iri))
        # add relation types
        relation_type_preflabel = postulates.at[i, "relations_types"].strip()
        relation_type_iri = utils.create_iri(f"Relation type: {relation_type_preflabel}", namespace="https://www.foom.com/core")
        kg = add_relation_triples(kg, postulate_iri, relation_type_iri, relation_type_preflabel, relation_class, relation_iri)
        # add links between relation types and concepts
        kg = add_relation_concept_triples(kg, postulates, "relations_types", relation_type_iri)

    return kg

def add_operations(kg: rdflib.Graph,
                   postulates: dict) -> rdflib.Graph:
    for i in postulates.index:
        postulate = postulates.at[i, "postulate_name"].strip()
        postulate_iri = utils.create_iri(postulate, namespace="https://www.foom.com/core")

    for postulate, postulate_data in postulates.items():
        postulate_iri = utils.create_iri(postulate, namespace="https://www.foom.com/core")
        for operation in postulate_data["operations"]:
            # add operation instance to class "Operation instance"
            operation_iri = utils.create_iri(f"Operation: {operation}", namespace="https://www.foom.com/core")
            kg.add((operation_iri, rdf_type, owl_individual))
            kg.add((operation_iri, rdf_type, operation_instance_class))
            kg.add((operation_iri, rdfs_label, rdflib.Literal(f"Operation: {operation}")))
            kg.add((operation_iri, skos_prefLabel, rdflib.Literal(operation)))
            kg.add((postulate_iri, refers_to, operation_iri))
            kg.add((operation_iri, is_used_in, postulate_iri))

            # add operation to class Operation and links them to domains and ranges
            for operation_type in postulate_data["operations_types"]:
                pass


    return kg

def postulate_links_to_concepts(kg: rdflib.Graph,
                                postulates: pd.DataFrame) -> rdflib.Graph:
    for i in postulates.index:
        postulate = postulates.at[i, "postulate_name"].strip()
        postulate_iri = utils.create_iri(postulate, namespace="https://www.foom.com/core")
        concepts_list = [concept.strip() for concept in postulates.at[i, "concepts"].split(",")]
        for concept in concepts_list:
            concept_iri = utils.create_iri(f"Concept: {concept}", namespace="https://www.foom.com/core")
            kg = concepts.add_triples(kg, concept, concept_iri, concept_class, "Concept")
            kg.add((postulate_iri, refers_to, concept_iri))
            kg.add((concept_iri, is_used_in, postulate_iri))
    return kg

def add_postulate_triples(kg: rdflib.Graph,
                          postulates: pd.DataFrame) -> rdflib.Graph:
    for i in postulates.index:
        postulate = postulates.at[i, "postulate_name"].strip()
        postulate_iri = utils.create_iri(postulate, namespace="https://www.foom.com/core")
        postulate_label = rdflib.Literal(postulate.replace("_", " "))
        kg.add((postulate_iri, rdfs_label, rdflib.Literal(postulate_label)))
        kg.add((postulate_iri, rdf_type, owl_individual))
        kg.add((postulate_iri, rdf_type, utils.create_iri("Postulate", namespace="https://www.foom.com/core")))
        kg.add((postulate_iri, rdfs_label, postulate_label))
        kg.add((postulate_iri, skos_prefLabel, postulate_label))
    return kg



# input_file_path = "input/ontology_postulates.txt"
input_file_path = "input/Euclid.Postulates.Book1.csv"
kg = add_postulates(kg, input_file_path)

# output ontology
utils.output_ontology(kg, "output", "euclid_book1.ttl", "turtle")

In [ ]:
# """
# Change here.

# Relation instance should refer to subrelations and to concepts used as arguments.

# Replace refers_to with has_component.
# """

#             # add relations as elements of Relation and links them to their domains
#             for relation_type in postulate_data["relations_types"]:
#                 relation_preflabel = re.match(r"(\w+)\(", relation_type).group(1)
#                 relation_type_iri = utils.create_iri(relation_preflabel, namespace="https://www.foom.com/core")
#                 # relation as elements of Relation
#                 kg.add((relation_type_iri, rdf_type, relation_class))
#                 kg.add((relation_type_iri, rdfs_label, rdflib.Literal(relation_type)))
#                 kg.add((relation_type_iri, skos_prefLabel, rdflib.Literal(relation_preflabel)))
#                 kg.add((relation_iri, refers_to, relation_type_iri))
#                 # domain
#                 domains = {domain.strip() for domain in re.match(r".*\((.*?)\)", relation_type).group(1).split(",")}
#                 for domain in domains:
#                     domain_iri = utils.create_iri(domain, namespace="https://www.foom.com/core")
#                     kg.add((relation_type_iri, has_domain, domain_iri))
#                     kg.add((domain_iri, rdf_type, domain_class))

#     return kg



# def organize_postulates(lines: list) -> dict:
#     postulates = {
#         line.split()[0]: {
#             "concepts": [],
#             "relations": [],
#             "relations_types": [],
#             "operations": [],
#             "operations_types": []
#         }
#         for line in lines
#     }
#     for line in lines:
#         items = [item.replace(";", "").strip() for item in line.split()]
#         for item in items[2:]:
#             postulates[items[0]][items[1]].append(item)
#     return postulates